In [13]:
from pathlib import Path

file = Path('data/stl/020_l.stl')
is_plot = True

## Load landmarks labelling

In [16]:
import pandas as pd

df = pd.read_pickle(file.with_suffix('.pkl'))
df

coord,x,y,z
landmark,,,
P1,69.653463,54.673492,-7.735388
P10,-168.788818,79.678271,-11.032859
P11,-122.558701,102.675117,-60.435042
P12,-123.716329,31.840291,-67.462064
P2,59.080175,94.482407,-9.113344
P3,18.191420,5.494654,-10.807760
P4,19.126142,99.146723,-18.009934
P5,-15.352643,2.470859,-11.472470
P6,-34.886927,64.674735,-62.592272


## Local frame

In [18]:
import pyvista as pv
from measure import frame
from mesh4d.analyse import crave

mesh = crave.fix_pvmesh_disconnect(pv.read(file))

# foot bottom cropped for estimating foot local frame
if is_plot:
    frame.plantar_clip(mesh, df).plot()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86805feb50_0&reconnect=auto' style='widt…

In [19]:
from measure import visual

axes_frame, origin = frame.estimate_foot_frame(mesh, df)

if is_plot:
    visual.plot_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f8682355fa0_1&reconnect=auto' style='widt…

In [20]:
mesh_clip = frame.foot_clip(mesh, df)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)

if is_plot:
    mesh_local.plot()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f8681533c10_2&reconnect=auto' style='widt…

In [21]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord,x,y,z
landmark,,,
P1,95.204932,-0.753839,6.982687
P10,-144.178050,-12.993784,13.050465
P11,-98.644717,-39.191606,61.496780
P12,-95.904574,31.487068,69.578191
P2,82.524019,-39.955482,7.914767
P3,46.503865,51.058825,11.454183
P4,42.489928,-42.608438,17.266966
P5,13.181717,55.865117,12.604671
P6,-9.036342,-5.981267,63.057902


## Metrics

In [33]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [34]:
from measure import metric

results.append(
    {
        'file': str(file),
        'FL': metric.fl(df_local),
        'MBL': metric.mbl(df_local),
        'LBL': metric.lbl(df_local),
        'ABW': metric.abw(df_local),
        'OBW': metric.obw(df_local),
        'OHW': metric.ohw(df_local),
        'BH': metric.bh(df_local),
        'IH': metric.ih(df_local),
        'BA': metric.ba(df_local),
        'T1A': metric.t1a(df_local),
        'T5A': metric.t5a(df_local),
        'ABG': metric.abg(df_local, mesh_local),
        'IG': metric.ig(df_local, mesh_local),
    }
)

In [35]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
data/stl/020_l.stl,239.382982,186.667978,157.359766,102.848186,93.667263,63.45343,63.057902,72.074679,71.421904,17.873216,21.37628,250.609346,254.362386


### Visualization

In [38]:
if is_plot:
    settings = {
        'FL': ['P1', 'P10', 'x'],
        'MBL': ['P4', 'P10', 'x'],
        'LBL': ['P5', 'P10', 'x'],
        'OBW': ['P4', 'P3', 'y'],
        'OHW': ['P9', 'P8', 'y'],
    }

    for name, [landmark1, landmark2, axis] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist_along_axis(scene, df_local, landmark1, landmark2, axis, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f867183f3d0_3&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f8694f57a90_4&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86814e1fd0_5&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f8694f57b80_6&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f8671827c70_7&reconnect=auto' style='widt…

In [39]:
if is_plot:
    settings = {
        'BH': 'P6',
        'IH': 'P7',
    }

    for name, landmark in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_height(scene, df_local, landmark, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f8682ab7f40_8&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86934cc850_9&reconnect=auto' style='widt…

In [40]:
if is_plot:
    settings = {
        'BA': ['P4', 'P5', 'P8'],
        'T1A': ['P4', 'P2', 'P8'],
        'T5A': ['P5', 'P3', 'P9'],
    }

    for name, [landmark_origin, landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_angle(scene, df_local, landmark_origin, landmark1, landmark2, actue_angle=True, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f862078dcd0_10&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86a282ce50_11&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86717f53a0_12&reconnect=auto' style='wid…

In [41]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_2landmarks(scene, df_local, mesh_local, ['P4', 'P5'], tangent_axis='z', name='ABG')
    scene.show()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86207bd4c0_13&reconnect=auto' style='wid…

In [42]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmark(scene, df_local, mesh_local, 'P6', 'x', name='IG')
    scene.show()

Widget(value="<iframe src='http://localhost:60715/index.html?ui=P_0x7f86207bd040_14&reconnect=auto' style='wid…